In [2]:
import numpy as np
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.preprocessing import image

In [3]:
#Создание генератора
datagen = ImageDataGenerator(rescale=1. / 255)

#Загрузка изображений
train_generator = datagen.flow_from_directory(
    'data/training',
    target_size=(80, 80),
    batch_size=30,
    class_mode='categorical')

val_generator = datagen.flow_from_directory(
    'data/valid',
    target_size=(80, 80),
    batch_size=30,
    class_mode='categorical')

test_generator = datagen.flow_from_directory(
    'data/test',
    target_size=(80, 80),
    batch_size=30,
    class_mode='categorical')



Found 13220 images belonging to 9 classes.
Found 13220 images belonging to 9 classes.
Found 255 images belonging to 9 classes.


In [4]:
#Создание нейронной сети
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(80, 80, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(9))
model.add(Activation('sigmoid'))


In [5]:
#Компиляция нейронной сети
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [6]:
#Обучеие нейронной сети

# Размер мини-выборки
batch_size = 30
# Количество изображений для обучения
nb_train_samples = 13220
# Количество изображений для проверки
nb_validation_samples = 13220
# Количество изображений для тестирования
nb_test_samples = 255

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=9,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)


Epoch 1/9
440/440 [==============================] - 782s 2s/step - loss: 0.6034 - acc: 0.8070 - val_loss: 0.0591 - val_acc: 0.9820
Epoch 2/9
440/440 [==============================] - 614s 1s/step - loss: 0.1654 - acc: 0.9512 - val_loss: 0.0421 - val_acc: 0.9872
Epoch 3/9
440/440 [==============================] - 577s 1s/step - loss: 0.0994 - acc: 0.9724 - val_loss: 0.0308 - val_acc: 0.9927
Epoch 4/9
440/440 [==============================] - 594s 1s/step - loss: 0.0668 - acc: 0.9794 - val_loss: 0.0188 - val_acc: 0.9956
Epoch 5/9
440/440 [==============================] - 580s 1s/step - loss: 0.0619 - acc: 0.9806 - val_loss: 0.0153 - val_acc: 0.9952
Epoch 6/9
440/440 [==============================] - 567s 1s/step - loss: 0.0509 - acc: 0.9842 - val_loss: 0.0112 - val_acc: 0.9962
Epoch 7/9
440/440 [==============================] - 589s 1s/step - loss: 0.0440 - acc: 0.9857 - val_loss: 0.0117 - val_acc: 0.9964
Epoch 8/9
440/440 [==============================] - 578s 1s/step - loss: 0.

In [8]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))


img_path = 'Test_img_ped.jpg'
img = image.load_img(img_path, target_size=(80, 80))

# Преобразуем изображением в массив numpy
x = image.img_to_array(img)

# Инвертируем и нормализуем изображение
x = 255 - x
x /= 255
x = np.expand_dims(x, axis=0)
prediction = model.predict(x)
#print(np.argmax(prediction))



Аккуратность на тестовых данных: 94.58%


In [20]:
# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("model_gen_j.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_gen_h.h5")
